In [1]:
from configure import configuration
from analysis import analysis
from generate import generate
from compare import compare

/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Global Healthy and Sustainable City Indicators (GHSCI) analysis for Las Palmas de Gran Canaria, Spain

This notebook contains an example of how the GHSCI tool can be used to run an analysis for a study region of interest, for example, a city or set of neighbourhoods.

In [2]:
codename = 'example_ES_Las_Palmas_2023'

In [3]:
configuration(codename)

Creating project configuration files, if not already existing in the
configuration folder...
	- config.yml exists.
	- datasets.yml exists.
	- indicators.yml exists.
	- osm_destination_definitions.csv exists.
	- osm_open_space.yml exists.
	- policies.yml exists.
	- _report_configuration.xlsx exists.

Configuration file for the specified study region codename
'example_ES_Las_Palmas_2023' already exists:
configuration/regions/example_ES_Las_Palmas_2023.yml.

Please open and edit this file in a text editor following the provided example
directions in order to complete configuration for your study region.  A
completed example study region configuration can be viewed in the file
'configuration/regions/example_ES_Las_Palmas_2023.yml'.

To view additional guidance on configuration, run this script again without a
codename.

Once configuration has been completed, to proceed to analysis for this city,
enter:
analysis example_ES_Las_Palmas_2023


In [4]:
analysis(codename)


Las Palmas de Gran Canaria (example_ES_Las_Palmas_2023)

Output directory:
  process/data/_study_region_outputs/example_ES_Las_Palmas_2023

The copy of region and project parameters from a previous analysis dated
2023-05-11 at 1019 saved in the output directory as
_parameters_2023-05-11_1019.yml matches the current configuration parameters and
will be retained.


Analysis time zone: Australia/Melbourne (to set time zone for where you are,
edit config.yml)

Analysis start:	2023-05-11_1818


Aggregate region summary analyses:  100%|██████████████████████████████| (13/13)

Analysis end:	2023-05-11_1819 (approximately 0.6 minutes)

To generate resources (data files, documentation, maps, figures, reports) using
the processed results for this study region, enter:

generate example_ES_Las_Palmas_2023

The Postgis SQL database for this city example_es_las_palmas_2023 can also be
accessed from QGIS or other applications by specifying the server as 'localhost'
and port as '5433', with username 'postgres' and password 'ghscic'.The SQL
database can also be explored on the command line by using the above password
after entering,'psql -U postgres -h gateway.docker.internal -p 5433 -d
"example_es_las_palmas_2023"'. When using psql, you can type '\dt' to list
database tables, '\d <table_name>' to list table columns, and 'SELECT * FROM
<table_name> LIMIT 10;' to view the first 10 rows of a table.  To exit psql,
enter '\q'.



In [5]:
generate(codename)


Las Palmas de Gran Canaria (example_ES_Las_Palmas_2023)

Output directory:
  process/data/_study_region_outputs/example_ES_Las_Palmas_2023

Analysis parameter summary text file
  _parameters.yml

Analysis log text file
  __Las Palmas de Gran Canaria__example_ES_Las_Palmas_2023_processing_log.txt

Data files
  example_ES_Las_Palmas_2023_1600m_buffer.gpkg
    - example_ES_Las_Palmas_2023_region
    - example_ES_Las_Palmas_2023_grid_100m
    - example_ES_Las_Palmas_2023_sample_points
    - aos_public_osm
    - dest_type
    - destinations
    - clean_intersections_12m
    - edges
    - nodes
    - pt_stops_headway
example_ES_Las_Palmas_2023_region.csv
example_ES_Las_Palmas_2023_grid_100m.csv

Data dictionaries
  output_data_dictionary.csv
  output_data_dictionary.xlsx

Metadata
  example_ES_Las_Palmas_2023_metadata.yml
  example_ES_Las_Palmas_2023_metadata.xml

Figures and maps (English)
  figures/access_profile_English.jpg
  figures/all_cities_walkability_English.jpg
  figures/pct_acces

In [6]:
compare(codename,'gtfs_ES_Las_Palmas_2023')


Las Palmas de Gran Canaria (example_ES_Las_Palmas_2023)

Output directory:
  process/data/_study_region_outputs/example_ES_Las_Palmas_2023

                   example_ES_Las_Palmas_2023  gtfs_ES_Las_Palmas_2023
pop_walkability                      0.650780                 0.650780
local_walkability                   -0.806465                -0.806465

Comparison saved as compare_example_ES_Las_Palmas_2023_gtfs_ES_Las_Palmas_2023_2023-05-11_1818.csv

